In [2]:
import sys
import pandas as pd
import os
sys.path.append('../')

import src.module_process_radiative as proc_rad
import src.module_process_energies as proc_ener
import src.module_miscellaneous as misc

In [3]:
def read_database_energy_levels(dbfilepath, terms):

    df = pd.read_csv(dbfilepath, sep='\t', usecols=[3,4,5,6,7,8])
    df = df.rename(columns={'LevelNumber': 'LevNum', 'SublevelNumber': 'SubLevNum'})

    copy_terms = terms.copy()
    copy_terms['LevNum'] = copy_terms.index.tolist()

    df = pd.merge(df, copy_terms, on='LevNum', how='inner')
    df['gi'] = df['2J'] + 1
    df['Energy(Ryd)'] = df['ExcitationWaven'] * 9.112670452E-6 # Rydberg
    df = df.drop(columns=['commentid', 'StatisticalWeight', '2J', 'ExcitationWaven'])
    
    df = df[['LevNum', 'SubLevNum', 'Conf', 'Term', 'S', 'L', 'gi', 'Energy(Ryd)']]

    df.index = df.index + 1
    return df

In [4]:
def leveltran_to_termtran(df_tranlevs, as_levs, as_terms):

    # creo un dataframe de donde voy a extraer las transiciones con los mismos terminos up y low
    df_copy = df_tranlevs.copy()

    df_copy.insert(0, 'Tk', None)
    df_copy.insert(1, 'Ti', None)

    # inserto el index (autostructure) de los términos en el dataframe de las transiciones para luego combinarlos
    for idx in df_copy.index:
        k = df_copy.loc[idx]['k']
        i = df_copy.loc[idx]['i']
        tk, ti = as_levs.loc[as_levs['K'] == k]['T'][k], as_levs.loc[as_levs['K'] == i]['T'][i]
        df_copy.at[idx, 'Tk'] = tk
        df_copy.at[idx, 'Ti'] = ti

    # calculo el número de transiciones con única combinación Tk, Ti
    pairs = [(x, y) for x, y in zip(df_copy['Tk'].tolist(), df_copy['Ti'].tolist())]
    ntran = len(set(pairs))

    tran_not_found = []

    # creo un nuevo dataframe con transiciones entre términos calculadas a partir de transiciones entre niveles
    df_terms = pd.DataFrame(columns=df_tranlevs.columns)

    for i in range(ntran):

        Tk, Ti = df_copy.iloc[0]['Tk'], df_copy.iloc[0]['Ti']
        rows_tran = df_copy.loc[(df_copy['Tk'] == Tk) & (df_copy['Ti'] == Ti)]
        df_copy = df_copy.drop(rows_tran.index)

        row_ener_Tk = as_levs.loc[as_levs['T'] == Tk]
        row_ener_Ti = as_levs.loc[as_levs['T'] == Ti]
        gL, gU = (2 * row_ener_Ti['J'] + 1).sum(), (2 * row_ener_Tk['J'] + 1).sum()

        idxs = rows_tran.index.tolist()
        idx = idxs[0]
        # rows_tran.at[idx, 'Aki'] = (rows_tran['Aki'] * rows_tran['gk']).sum() / gU
        rows_tran.at[idx, 'Aki'] = rows_tran['Aki'].sum()
        rows_tran.at[idx, 'gk'] = gU
        rows_tran.at[idx, 'gi'] = gL

        # find k and i index values from terms dataframe
        k_terms = as_terms.loc[as_terms['T'] == Tk]['K'].index.tolist()
        i_terms = as_terms.loc[as_terms['T'] == Ti]['K'].index.tolist()
        if (len(k_terms) > 0) and (len(i_terms) > 0):
            rows_tran.at[idx, 'k'] = k_terms[0]
            rows_tran.at[idx, 'i'] = i_terms[0]
        else:
            # hay términos que no están la database y tengo que droppear las transiciones que involucran esos términos
            tran_not_found.append((Tk, Ti))
            continue

        # drop all rows except first from transition dataframe
        if len(idxs) > 1:
            rows_tran = rows_tran.drop(index=idxs[1:])

        df_terms = pd.concat([df_terms, rows_tran])

    # eliminar transiciones entre el mismo término
    df_terms = df_terms.drop(df_terms.loc[df_terms['k'] == df_terms['i']].index)
    df_terms = df_terms.drop(columns=['Tk', 'Ti'])
    return df_terms

In [96]:
def compute_superlevels_Aki(df_tranterms, termdict_DB, terms_DB, nlevmin, nlevmax, debug):

    try:
        # insertar index de database
        df_tranterms.insert(0, 'k_db', None)
        df_tranterms.insert(1, 'i_db', None)
    except:
        pass

    for idx in df_tranterms.index:
        row = df_tranterms.loc[idx:idx]
        k, i = row['k'][idx], row['i'][idx]
        nk, ni = termdict_DB[k], termdict_DB[i]
        df_tranterms.loc[idx,'k_db'] = nk
        df_tranterms.loc[idx,'i_db'] = ni

    # compute superlines
    df_copy = df_tranterms.copy()
    df_suplevs = pd.DataFrame(columns=df_copy.columns)

    for i in range(nlevmin):
        for k in range(i + 1, nlevmax):

            L = i + 1
            U = k + 1

            # dataframe con los mismos niveles up y low
            df_k = df_copy.loc[df_copy['k_db'] == U]
            df = df_k.loc[df_k['i_db'] == L]

            k_list = df['k'].tolist()
            i_list = df['i'].tolist()

            ntran = len(df)
            A_UL, f_LU, gf = 0, 0, 0

            if ntran > 1:

                gk_list = set(df['gk'].tolist())
                gi_list = set(df['gi'].tolist())
                gU = df['gk'].sum() if len(gk_list) > 1 else list(set(gk_list))[0]
                gL = df['gi'].sum() if len(gi_list) > 1 else list(set(gi_list))[0]

                # A_UL = (row['Aki'] * row['gk'] / row['gi']).sum() 
                # A_UL = (row['Aki'] * row['gk']).sum() 
                A_UL = df['Aki'].sum()
                if 'Aki(NIST)' in df.columns: A_UL_NIST = df['Aki(NIST)'].sum()

                # for idx in df.index:
                #     row = df.loc[idx]
                #     gk, gi = row['gk'], row['gi']
                #     # A_UL += gk * row['Aki']
                #     A_UL += row['Aki']
                if debug: 
                    print("AS:", k_list, i_list)
                    print("DB:", U, L)
                    # print(df)
                    # print()
                    # print('A_UL = ', A_UL)
                    # print('A_UL(NIST) = ', A_UL_NIST)

                new_row = df.iloc[0:1]
                new_row = new_row.reset_index(drop=True)
                for col in new_row.columns:
                    if col not in ['k_db', 'i_db', 'k', 'i', 'Ek(Ry)', 'Ei(Ry)']:
                        new_row.at[0, col] = None

                for col in ['Conf', 'Term', 'S', 'L']: 
                    new_row.at[0, col+'k'] = terms_DB.loc[U][col]
                    new_row.at[0, col+'i'] = terms_DB.loc[L][col]

                new_row.at[0, 'gk'] = terms_DB.loc[U]['gi']
                new_row.at[0, 'Ek(Ry)'] = terms_DB.loc[U]['Energy(Ryd)']
                new_row.at[0, 'gi'] = terms_DB.loc[L]['gi']
                new_row.at[0, 'Ei(Ry)'] = terms_DB.loc[L]['Energy(Ryd)']
                new_row.at[0, 'Aki'] = A_UL
                if 'Aki(NIST)' in df.columns: 
                    new_row.at[0, 'Aki(NIST)'] = A_UL_NIST if A_UL_NIST != 0 else None
                    new_row.at[0, 'source'] = 'nist' if A_UL_NIST != 0 else 'auto'

                df_suplevs = pd.concat([df_suplevs, new_row])

                # if debug: 
                #     print(gL, gU, A_UL)
                #     print()
                #     print(new_row)
                #     print('=' * 25)

            elif ntran == 1:

                # if debug: 
                #     print(df)
                #     print('=' * 25)

                df_suplevs = pd.concat([df_suplevs, df])

    return df_suplevs

In [6]:
mainfolder = '~/magnesio'

In [7]:
folder = '../rad_ishftls/E3tran'
ncfgmax = 79
ncfgs = 85
nlevs = 339
nterms = 189
colname = "AS(Ryd)"

oic_pathfile = os.path.join(folder, "oic")
olg_pathfile = os.path.join(folder, "olg")

as_levs = proc_ener.read_levels_from_oic(oic_pathfile, ncfgs, nlevs, colname, ncfgmax=ncfgmax)
as_terms = proc_ener.read_terms_from_olg(olg_pathfile, nterms, colname, ncfgmax=ncfgmax)

In [8]:
len(as_terms), len(as_levs)

(158, 280)

In [9]:
# Read database (DB) values
nsuplev = 55
dbfilepath_terms = os.path.join(mainfolder, 'resumen/ModelAtomicIonLevel-IMPORTED.txt')
terms_DB = proc_ener.read_database_energies(dbfilepath_terms)

dbfilepath_levs = os.path.join(mainfolder, 'resumen/ModelAtomicIonLevelSublevel-IMPORTED.txt')
levels_DB = read_database_energy_levels(dbfilepath_levs, terms_DB)

compterms = {
    'database(NIST)': {
        'terms': terms_DB, 
        'suplev': nsuplev # => index where superlevels appear
    }
}

In [10]:
# relaciono terminos de autostructure con superlevels de database
nsuplev = 55
levdict_DB = misc.dict_levels_AS('LV', as_levs, levels_DB, nsuplev=nsuplev)
termdict_DB = misc.dict_levels_AS('T', as_terms, terms_DB, nsuplev=nsuplev)

=> warning: using superlevels predefined
... all done!

=> warning: using superlevels predefined
... all done!



In [11]:
# inserto en levs/terms dataframe la data de nist (de database)
proc_ener.insert_energy_column(as_levs, 'database(NIST)', levdict_DB, levels_DB)
as_levs = as_levs[as_levs['database(NIST)'].notnull()]

proc_ener.insert_energy_column(as_terms, 'database(NIST)', termdict_DB, terms_DB)
as_terms = as_terms[as_terms['database(NIST)'].notnull()]

In [12]:
len(as_terms), len(as_levs)

(124, 214)

In [13]:
try:
    df_tranlevs = pd.read_csv('AS_radtran_levels.dat')
    df_tranterms = pd.read_csv('AS_radtran_terms.dat')
except:
    # import level-to-level transition data from olg file
    ntranE1_levs = 13982
    E1_levs = proc_rad.read_transition_from_olg('levels', olg_pathfile, 'E1', ntranE1_levs, as_levs)
    db_E1levs = proc_rad.transformsymb_radtran('LV', E1_levs, as_levs)

    ntranE2_levs = 18159
    E2_levs = proc_rad.read_transition_from_olg('levels', olg_pathfile, 'E2', ntranE2_levs, as_levs)
    db_E2levs = proc_rad.transformsymb_radtran('LV', E2_levs, as_levs)

    ntranE3_levs = 13165
    E3_levs = proc_rad.read_transition_from_olg('levels', olg_pathfile, 'E3', ntranE3_levs, as_levs)
    db_E3levs = proc_rad.transformsymb_radtran('LV', E3_levs, as_levs)

    # import term-to-term transition data from olg file
    ntranE1_terms = 2767
    E1_terms = proc_rad.read_transition_from_olg('terms', olg_pathfile, 'E1', ntranE1_terms, as_terms)
    db_E1terms = proc_rad.transformsymb_radtran('T', E1_terms, as_terms)

    ntranE2_terms = 3162
    E2_terms = proc_rad.read_transition_from_olg('terms', olg_pathfile, 'E2', ntranE2_terms, as_terms)
    db_E2terms = proc_rad.transformsymb_radtran('T', E2_terms, as_terms)

    ntranE3_terms = 1335
    E3_terms = proc_rad.read_transition_from_olg('terms', olg_pathfile, 'E3', ntranE3_terms, as_terms)
    db_E3terms = proc_rad.transformsymb_radtran('T', E3_terms, as_terms)

    df_tranlevs = pd.concat([db_E1levs, db_E2levs, db_E3levs])
    df_tranterms = pd.concat([db_E1terms, db_E2terms, db_E3terms])

    df_tranlevs = df_tranlevs.reset_index(drop=True)
    df_tranterms = df_tranterms.reset_index(drop=True)

    df_tranlevs = df_tranlevs.drop(columns=['fik', 'gf'])
    df_tranterms = df_tranterms.drop(columns=['fik', 'gf'])

    df_tranlevs.to_csv('AS_radtran_levels.dat', index=False)
    df_tranterms.to_csv('AS_radtran_terms.dat', index=False)

In [14]:
df_tranlevs.head()

,k,Confk,Termk,Sk,Lk,Pk,gk,Ek(Ry),i,Confi,Termi,Si,Li,Pi,gi,Ei(Ry),Aki
0,3,3s.3p,3P*,3,1,1,3,0.199298,1,3s2,1S,1,0,0,1,0.000000,1.844390e+02
1,5,3s.3p,1P*,1,1,1,3,0.319411,1,3s2,1S,1,0,0,1,0.000000,4.367560e+08
2,6,3s.4s,3S,3,0,0,3,0.375418,2,3s.3p,3P*,3,1,1,1,0.199116,1.197850e+07
3,6,3s.4s,3S,3,0,0,3,0.375418,3,3s.3p,3P*,3,1,1,3,0.199298,3.585020e+07
4,6,3s.4s,3S,3,0,0,3,0.375418,4,3s.3p,3P*,3,1,1,5,0.199669,5.946640e+07


In [15]:
len(df_tranlevs), len(df_tranterms)

(15399, 2946)

# Compute T-to-T transitions from LV-to-LV transitions

In [16]:
filepath = os.path.join(folder, 'AS_radtran_termsfromlevs.dat')
try:
    df_tranterms_fromlevs = pd.read_csv(filepath)
except:
    df_tranterms_fromlevs = leveltran_to_termtran(df_tranlevs, as_levs, as_terms)
    df_tranterms_fromlevs = df_tranterms_fromlevs.drop(columns=['fik', 'gf'])
    df_tranterms_fromlevs.to_csv(filepath, index=False)

In [17]:
df_tranterms_fromlevs.head()

,k,Confk,Termk,Sk,Lk,Pk,gk,Ek(Ry),i,Confi,Termi,Si,Li,Pi,gi,Ei(Ry),Aki
0,2,3s.3p,3P*,3,1,1,9,0.199364,1,3s2,1S,1,0,0,1,0.000000,1.844395e+02
1,3,3s.3p,1P*,1,1,1,3,0.319411,1,3s2,1S,1,0,0,1,0.000000,4.367560e+08
2,4,3s.4s,3S,3,0,0,3,0.375418,2,3s.3p,3P*,3,1,1,9,0.199244,1.072951e+08
3,4,3s.4s,3S,3,0,0,3,0.375418,3,3s.3p,1P*,1,1,1,3,0.319411,1.055809e-01
4,5,3s.4s,1S,1,0,0,1,0.396432,2,3s.3p,3P*,3,1,1,9,0.199364,6.772053e+02


In [18]:
df_tranterms.head()

,k,Confk,Termk,Sk,Lk,Pk,gk,Ek(Ry),i,Confi,Termi,Si,Li,Pi,gi,Ei(Ry),Aki
0,3,3s.3p,1P*,1,1,1,3,0.319411,1,3s2,1S,1,0,0,1,0.000000,436757000.0
1,4,3s.4s,3S,3,0,0,3,0.375419,2,3s.3p,3P*,3,1,1,9,0.199116,107295000.0
2,5,3s.4s,1S,1,0,0,1,0.396432,3,3s.3p,1P*,1,1,1,3,0.319411,39649100.0
3,6,3s.3d,1D,1,2,0,5,0.422856,3,3s.3p,1P*,1,1,1,3,0.319411,26426600.0
4,7,3s.4p,3P*,3,1,1,9,0.435961,4,3s.4s,3S,3,0,0,3,0.375419,13641900.0


In [19]:
len(df_tranterms_fromlevs), len(df_tranterms)

(6000, 2946)

In [16]:
# pairs = [(x, y) for x, y in zip(df_tranterms_fromlevs['k'].tolist(), df_tranterms_fromlevs['i'].tolist())]
# lista = list(set(pairs))
# lista.sort()

# Include NIST

In [20]:
nistpath = os.path.join(mainfolder, "mg_xtra/NIST_files")
nist_tranlevs = proc_rad.read_NIST_radiative(nistpath)

In [21]:
df_tranlevs = proc_rad.include_NIST('E1', df_tranlevs, nist_tranlevs)

In [22]:
df_tranlevs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15399 entries, 0 to 15398
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   k          15399 non-null  int64  
 1   Confk      15399 non-null  object 
 2   Termk      15399 non-null  object 
 3   Sk         15399 non-null  int64  
 4   Lk         15399 non-null  int64  
 5   Pk         15399 non-null  int64  
 6   gk         15399 non-null  int64  
 7   Ek(Ry)     15399 non-null  float64
 8   i          15399 non-null  int64  
 9   Confi      15399 non-null  object 
 10  Termi      15399 non-null  object 
 11  Si         15399 non-null  int64  
 12  Li         15399 non-null  int64  
 13  Pi         15399 non-null  int64  
 14  gi         15399 non-null  int64  
 15  Ei(Ry)     15399 non-null  float64
 16  Aki        15399 non-null  float64
 17  Aki(NIST)  246 non-null    object 
 18  source     15399 non-null  object 
dtypes: float64(3), int64(10), object(6)
memory usa

## Compute NIST for terms-to-term transitions

In [33]:
nist_tranlevs['Confk-Termk'] = [' '.join([c, t]) for c, t in zip(nist_tranlevs['Confk'].tolist(), nist_tranlevs['Termk'].tolist())]
nist_tranlevs['Confi-Termi'] = [' '.join([c, t]) for c, t in zip(nist_tranlevs['Confi'].tolist(), nist_tranlevs['Termi'].tolist())]

In [43]:
pairs = [(x, y) for x, y in zip(nist_tranlevs['Confk-Termk'].tolist(), nist_tranlevs['Confi-Termi'].tolist())]
lista = list(set(pairs))

In [51]:
# for cktk, citi in lista:
#     print(cktk, '=>', citi)
#     df = nist_tranlevs.loc[(nist_tranlevs['Confk-Termk'] == cktk) & (nist_tranlevs['Confi-Termi'] == citi)]
#     if len(df) > 1:
#         print(df)
#         new_row = df.iloc[0]
#         print(new_row)
#         break

In [56]:
# print_tran = proc_rad.prep_print('E1', df_tranlevs)

In [26]:
df_tranlevs.tail()

,k,Confk,Termk,Sk,Lk,Pk,gk,Ek(Ry),i,Confi,Termi,Si,Li,Pi,gi,Ei(Ry),Aki,Aki(NIST),source
15394,280,3p2,1S,1,0,0,1,0.622168,217,3s.14f,3F*,3,3,1,5,0.538021,3.959210e-19,None,auto
15395,280,3p2,1S,1,0,0,1,0.622168,218,3s.14f,3F*,3,3,1,7,0.538021,2.568290e-03,None,auto
15396,280,3p2,1S,1,0,0,1,0.622168,230,3s.15f,1F*,1,3,1,7,0.540552,1.431420e-02,None,auto
15397,280,3p2,1S,1,0,0,1,0.622168,231,3s.15f,3F*,3,3,1,5,0.540552,1.941640e-19,None,auto
15398,280,3p2,1S,1,0,0,1,0.622168,232,3s.15f,3F*,3,3,1,7,0.540552,7.263790e-03,None,auto


# Calcular superlevels

In [90]:
df_tranlevs = df_tranlevs.reset_index(drop=True)

In [97]:
# transitions between levels
df_tranlevs_suplevs = compute_superlevels_Aki(df_tranlevs, levdict_DB, levels_DB, nlevmin=128, nlevmax=129, debug=True)
df_tranlevs_suplevs = df_tranlevs_suplevs.drop(columns=['Pk', 'Pi'])
df_tranlevs_suplevs = df_tranlevs_suplevs.reset_index(drop=True)

AS: [100, 102, 103, 108, 109, 110] [1, 1, 1, 1, 1, 1]
DB: 100 1
AS: [106, 112, 107] [1, 1, 1]
DB: 101 1
AS: [122, 124, 125, 130, 131, 132] [1, 1, 1, 1, 1, 1]
DB: 103 1
AS: [152, 154, 155, 160, 161, 162] [1, 1, 1, 1, 1, 1]
DB: 106 1
AS: [166, 171, 172, 174, 175, 176] [1, 1, 1, 1, 1, 1]
DB: 109 1
AS: [180, 185, 186, 188, 189, 190] [1, 1, 1, 1, 1, 1]
DB: 112 1
AS: [194, 199, 200, 202, 203, 204] [1, 1, 1, 1, 1, 1]
DB: 115 1
AS: [208, 213, 214, 216, 217, 218] [1, 1, 1, 1, 1, 1]
DB: 117 1
AS: [222, 224, 225, 230, 231, 232] [1, 1, 1, 1, 1, 1]
DB: 119 1
AS: [102, 109, 100, 103, 104, 113] [2, 2, 2, 2, 2, 2]
DB: 100 2
AS: [106, 107, 112] [2, 2, 2]
DB: 101 2
AS: [124, 131, 122, 125, 126, 135] [2, 2, 2, 2, 2, 2]
DB: 103 2
AS: [154, 161, 152, 155, 156] [2, 2, 2, 2, 2]
DB: 106 2
AS: [171, 175, 166, 172, 173] [2, 2, 2, 2, 2]
DB: 109 2
AS: [185, 189, 180, 186, 187] [2, 2, 2, 2, 2]
DB: 112 2
AS: [199, 203, 194, 200, 201] [2, 2, 2, 2, 2]
DB: 115 2
AS: [213, 217, 208, 214, 215] [2, 2, 2, 2, 2]
DB: 117 2


In [102]:
len(df_tranlevs), len(df_tranlevs_suplevs)

(15399, 5684)

In [100]:
df_tranlevs_suplevs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5684 entries, 0 to 5683
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   k_db       5684 non-null   object
 1   i_db       5684 non-null   object
 2   k          5684 non-null   object
 3   Confk      5684 non-null   object
 4   Termk      5684 non-null   object
 5   Sk         5684 non-null   object
 6   Lk         5684 non-null   object
 7   gk         5684 non-null   object
 8   Ek(Ry)     5684 non-null   object
 9   i          5684 non-null   object
 10  Confi      5684 non-null   object
 11  Termi      5684 non-null   object
 12  Si         5684 non-null   object
 13  Li         5684 non-null   object
 14  gi         5684 non-null   object
 15  Ei(Ry)     5684 non-null   object
 16  Aki        5684 non-null   object
 17  Aki(NIST)  246 non-null    object
 18  source     5684 non-null   object
dtypes: object(19)
memory usage: 843.8+ KB


In [101]:
filepath = os.path.join(folder, 'AS_radtran_levels-suplev.dat')
df_tranlevs_suplevs.to_csv(filepath, index=False)

In [103]:
# transitions between terms
df_tranterms_suplevs = compute_superlevels_Aki(df_tranterms, termdict_DB, terms_DB, nlevmin=84, nlevmax=85, debug=False)
df_tranterms_suplevs = df_tranterms_suplevs.drop(columns=['Pk', 'Pi'])

In [104]:
len(df_tranterms), len(df_tranterms_suplevs)

(2946, 1755)

In [105]:
df_tranterms_suplevs.head()

,k_db,i_db,k,Confk,Termk,Sk,Lk,gk,Ek(Ry),i,Confi,Termi,Si,Li,gi,Ei(Ry),Aki
0,3,1,3,3s.3p,1P*,1,1,3,0.319411,1,3s2,1S,1,0,1,0.0,436757000.0
1183,6,1,6,3s.3d,1D,1,2,5,0.422856,1,3s2,1S,1,0,1,0.0,2367.74
7,9,1,9,3s.4p,1P*,1,1,3,0.449681,1,3s2,1S,1,0,1,0.0,140898000.0
1190,12,1,12,3s.4d,1D,1,2,5,0.484199,1,3s2,1S,1,0,1,0.0,906.519
2583,15,1,15,3s.4f,1F*,1,3,7,0.498249,1,3s2,1S,1,0,1,0.0,0.001858


In [106]:
df_tranterms_suplevs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1755 entries, 0 to 1182
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   k_db    1755 non-null   object
 1   i_db    1755 non-null   object
 2   k       1755 non-null   object
 3   Confk   1755 non-null   object
 4   Termk   1755 non-null   object
 5   Sk      1755 non-null   object
 6   Lk      1755 non-null   object
 7   gk      1755 non-null   object
 8   Ek(Ry)  1755 non-null   object
 9   i       1755 non-null   object
 10  Confi   1755 non-null   object
 11  Termi   1755 non-null   object
 12  Si      1755 non-null   object
 13  Li      1755 non-null   object
 14  gi      1755 non-null   object
 15  Ei(Ry)  1755 non-null   object
 16  Aki     1755 non-null   object
dtypes: object(17)
memory usage: 246.8+ KB


In [107]:
filepath = os.path.join(folder, 'AS_radtran_terms-suplev.dat')
df_tranterms_suplevs.to_csv(filepath, index=False)

In [108]:
# transitions between terms (computed from levels)
df_tranterms_fromlevs_suplevs = compute_superlevels_Aki(df_tranterms_fromlevs, termdict_DB, terms_DB, nlevmin=84, nlevmax=85, debug=False)
df_tranterms_fromlevs_suplevs = df_tranterms_fromlevs_suplevs.sort_values(by=['k_db'])
df_tranterms_fromlevs_suplevs = df_tranterms_fromlevs_suplevs.drop(columns=['Pk', 'Pi'])

In [109]:
len(df_tranterms_fromlevs), len(df_tranterms_fromlevs_suplevs)

(6000, 3001)

In [110]:
df_tranterms_fromlevs_suplevs.head()

,k_db,i_db,k,Confk,Termk,Sk,Lk,gk,Ek(Ry),i,Confi,Termi,Si,Li,gi,Ei(Ry),Aki
0,2,1,2,3s.3p,3P*,3,1,9,0.199364,1,3s2,1S,1,0,1,0.0,184.439451
1,3,1,3,3s.3p,1P*,1,1,3,0.319411,1,3s2,1S,1,0,1,0.0,436756000.0
2546,3,2,3,3s.3p,1P*,1,1,3,0.319411,2,3s.3p,3P*,3,1,9,0.199244,0.000383
2,4,2,4,3s.4s,3S,3,0,3,0.375418,2,3s.3p,3P*,3,1,9,0.199244,107295100.000196
3,4,3,4,3s.4s,3S,3,0,3,0.375418,3,3s.3p,1P*,1,1,3,0.319411,0.105581


In [111]:
filepath = os.path.join(folder, 'AS_radtran_termsfromlevs-suplev.dat')
df_tranterms_fromlevs_suplevs.to_csv(filepath, index=False)

In [113]:
df_tranterms_fromlevs_suplevs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3001 entries, 0 to 2545
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   k_db    3001 non-null   object
 1   i_db    3001 non-null   object
 2   k       3001 non-null   object
 3   Confk   3001 non-null   object
 4   Termk   3001 non-null   object
 5   Sk      3001 non-null   object
 6   Lk      3001 non-null   object
 7   gk      3001 non-null   object
 8   Ek(Ry)  3001 non-null   object
 9   i       3001 non-null   object
 10  Confi   3001 non-null   object
 11  Termi   3001 non-null   object
 12  Si      3001 non-null   object
 13  Li      3001 non-null   object
 14  gi      3001 non-null   object
 15  Ei(Ry)  3001 non-null   object
 16  Aki     3001 non-null   object
dtypes: object(17)
memory usage: 422.0+ KB


In [29]:
len(df_tranterms_fromlevs_suplevs), len(df_tranterms_suplevs)

(3001, 1755)

In [33]:
df = pd.read_csv(os.path.join(folder, 'AS_radtran_termsfromlevs-suplev.dat'))